## Image and Text Embeddings Inference using Batch Endpoints

This sample shows how to deploy `embeddings` type models to a batch endpoint for image and text embeddings inference.

### Task
`embeddings` takes in images and/or text samples. For each image and text sample, feature embeddings are returned from the model.
 
### Model
Models that can perform the `embeddings` task are tagged with `embeddings`. We will use the `OpenAI-CLIP-Image-Text-Embeddings-vit-base-patch32` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import_model_into_registry.ipynb) and then use them for inference. 

### Inference data
We will use the [fridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-classification/fridgeObjects.zip) dataset.


### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Prepare data for inference - Using a folder of CSV files
4. Deploy the model to a batch endpoint
5. Test the endpoint - Using a folder of CSV files
6. Clean up resources - delete the endpoint

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)
# Generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

#### Create a compute cluster
Use the model card from the AzureML system registry to check the minimum required inferencing SKU, referenced as size below. If you already have a sufficient compute cluster, you can simply define the name in compute_name in the following code block.

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = workspace_ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        description="An AML compute cluster",
        size="Standard_DS3_V2",
        min_instances=0,
        max_instances=3,
        idle_time_before_scale_down=120,
    )
    workspace_ml_client.begin_create_or_update(compute_config).result()

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `embeddings` task. In this example, we use the `OpenAI-CLIP-Image-Text-Embeddings-vit-base-patch32` model. If you have opened this notebook for a different model, replace the model name accordingly.

In [ ]:
model_name = "OpenAI-CLIP-Image-Text-Embeddings-vit-base-patch32"

foundation_model = registry_ml_client.models.get(name=model_name, label="latest")
print(
    f"\n\nUsing model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for inferencing"
)

### 3. Prepare data for inference - Using a folder of csv files

We will use the [fridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-classification/fridgeObjects.zip) dataset for multi-class classification. The fridge object dataset is stored in a directory. There are four different folders inside:
- /water_bottle
- /milk_bottle
- /carton
- /can



In [ ]:
import os
import urllib
import shutil
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# Download data
download_url = "https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-classification/fridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# Extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")

# Delete zip file
os.remove(data_file)

#### 3.1 Prepare image data for batch inference

In [ ]:
import base64


def read_image(image_path):
    with open(image_path, "rb") as f:
        data = f.read()
        data = base64.encodebytes(data).decode("utf-8")
        return data


image_list = []
image_path_list = []

for dir_name in os.listdir(dataset_dir):
    dir_path = os.path.join(dataset_dir, dir_name)
    for path, _, files in os.walk(dir_path):
        for file in files:
            image_path = os.path.join(path, file)
            image = read_image(image_path)
            image_path_list.append(image_path)
            image_list.append(image)

#### 3.2 Prepare a folder of CSV files with base64 images for batch inference input

We can provide input images to batch inference in a csv file containing a column named "image" having either images in base64 format or publicly accessible image URLs. We provide empty string in the same csv file in the column named "text".

The deployment in the `Create batch deployment` section below takes the argument `mini_batch_size`, which is the number of CSV files processed by the model in a single mini_batch. To limit the number of images processed in each mini_batch we split the dataset into multiple csv files.

In [ ]:
import base64
import pandas as pd
from pathlib import Path

image_csv_folder_path = os.path.join(dataset_parent_dir, "batch_image")
os.makedirs(image_csv_folder_path, exist_ok=True)
batch_input_file = "batch_input.csv"

data = [[image, ""] for image in image_list]
batch_df = pd.DataFrame(data, columns=["image", "text"])

In [ ]:
# Divide this into files of 10 rows each
batch_size_per_predict = 10
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(os.path.join(image_csv_folder_path, str(i) + batch_input_file))

# Check out the first and last file name created
input_paths = sorted(Path(image_csv_folder_path).iterdir(), key=os.path.getmtime)
input_files = [os.path.basename(path) for path in input_paths]
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "milk_bottle", "99.jpg")
Image(filename=sample_image)

#### 3.3 Prepare a folder of CSV files with text samples for batch inference input

We can provide text samples to batch inference in a csv file containing a column named "text". We provide empty strings in the same csv file in the column named "image".

The deployment in the `Create batch deployment` section below takes the argument `mini_batch_size`, which is the number of CSV files processed by the model in a single mini_batch. To limit the number of images processed in each mini_batch we split the dataset into multiple csv files.

In [ ]:
import base64
import pandas as pd
from pathlib import Path
import random
import string

text_csv_folder_path = os.path.join(dataset_parent_dir, "batch_text")
os.makedirs(text_csv_folder_path, exist_ok=True)
batch_input_file = "batch_input.csv"

data = [
    ["", "a photo of a " + os.path.basename(os.path.dirname(image_path))]
    for image_path in image_path_list
]
batch_df = pd.DataFrame(data, columns=["image", "text"])

In [ ]:
# Divide this into files of 10 rows each
batch_size_per_predict = 10
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(os.path.join(text_csv_folder_path, str(i) + batch_input_file))

# Check out the first and last file name created
input_paths = sorted(Path(text_csv_folder_path).iterdir(), key=os.path.getmtime)
input_files = [os.path.basename(path) for path in input_paths]
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

#### 3.4 Prepare a folder of CSV files with with base64 images and text samples for batch inference input

We can provide input images to batch inference in a csv file containing a column named "image" having either images in base64 format or publicly accessible image URLs. We provide text sample strings in the same csv file in the column named "text".

The deployment in the `Create batch deployment` section below takes the argument `mini_batch_size`, which is the number of CSV files processed by the model in a single mini_batch. To limit the number of images processed in each mini_batch we split the dataset into multiple csv files.

In [ ]:
import base64
import pandas as pd
from pathlib import Path
import random
import string

image_text_csv_folder_path = os.path.join(dataset_parent_dir, "batch_image_text")
os.makedirs(image_text_csv_folder_path, exist_ok=True)
batch_input_file = "batch_input.csv"

data = [
    [image_list[i], "a photo of a " + os.path.basename(os.path.dirname(image_path))]
    for i in range(len(image_list))
]
batch_df = pd.DataFrame(data, columns=["image", "text"])

In [ ]:
# Divide this into files of 10 rows each
batch_size_per_predict = 10
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(
        os.path.join(image_text_csv_folder_path, str(i) + batch_input_file)
    )

# Check out the first and last file name created
input_paths = sorted(Path(image_text_csv_folder_path).iterdir(), key=os.path.getmtime)
input_files = [os.path.basename(path) for path in input_paths]
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

### 4. Deploy the model to a batch endpoint
Batch endpoints are endpoints that are used to do batch inferencing on large volumes of data over a period of time. The endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis. For more information on batch endpoints and deployments see [What are batch endpoints?](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2#what-are-batch-endpoints).

* Create a batch endpoint.
* Create a batch deployment.
* Set the deployment as default; doing so allows invoking the endpoint without specifying the deployment's name.

#### Create a batch endpoint

In [ ]:
import time, sys
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    BatchRetrySettings,
    AmlCompute,
)

# Endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
endpoint_name = "clip-embeddings-batch" + str(timestamp)
# Create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Batch endpoint for "
    + foundation_model.name
    + ", for image-text-embeddings task",
)
workspace_ml_client.begin_create_or_update(endpoint).result()

#### Create a batch deployment

In [ ]:
deployment_name = "embed-clip-vit-base-patch32"

deployment = BatchDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=foundation_model.id,
    compute=compute_name,
    error_threshold=0,
    instance_count=1,
    logging_level="info",
    max_concurrency_per_instance=1,
    mini_batch_size=2,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=600),
)
workspace_ml_client.begin_create_or_update(deployment).result()

#### Set the deployment as default

In [ ]:
endpoint = workspace_ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment_name
workspace_ml_client.begin_create_or_update(endpoint).result()

print(f"The default deployment is {endpoint.defaults.deployment_name}")

### 5.1 Test the endpoint - Using a folder of CSV files with base64 images

Invoke the batch endpoint with the input parameter pointing to the folder of CSV files containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

Note: If job failed with Out of Memory Error then please try splitting your input into smaller csv files or decreasing `mini_batch_size` for the deployment.

In [ ]:
job = None
input = Input(path=image_csv_folder_path, type=AssetTypes.URI_FOLDER)
num_retries = 3
for i in range(num_retries):
    try:
        job = workspace_ml_client.batch_endpoints.invoke(
            endpoint_name=endpoint.name, input=input
        )
        break
    except Exception as e:
        if i == num_retries - 1:
            raise e
        else:
            print("Endpoint invocation failed. Retrying after 5 seconds...")
            time.sleep(5)
if job is not None:
    workspace_ml_client.jobs.stream(job.name)

In [ ]:
scoring_job = list(workspace_ml_client.jobs.list(parent_job_name=job.name))[0]

workspace_ml_client.jobs.download(
    name=scoring_job.name,
    download_path=os.path.join(dataset_parent_dir, "csv-output"),
    output_name="score",
)

predictions_file = os.path.join(
    dataset_parent_dir, "csv-output", "named-outputs", "score", "predictions.csv"
)

# Load the batch predictions file with no headers into a dataframe and set your column names
score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=["row_number_per_file", "image_features", "file_name"],
)
score_df.head()

### 5.2 Test the endpoint - Using a folder of CSV files with text samples

Invoke the batch endpoint with the input parameter pointing to the folder of CSV files containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

Note: If job failed with Out of Memory Error then please try splitting your input into smaller csv files or decreasing `mini_batch_size` for the deployment.

In [ ]:
job = None
input = Input(path=text_csv_folder_path, type=AssetTypes.URI_FOLDER)
num_retries = 3
for i in range(num_retries):
    try:
        job = workspace_ml_client.batch_endpoints.invoke(
            endpoint_name=endpoint.name, input=input
        )
        break
    except Exception as e:
        if i == num_retries - 1:
            raise e
        else:
            print("Endpoint invocation failed. Retrying after 5 seconds...")
            time.sleep(5)
if job is not None:
    workspace_ml_client.jobs.stream(job.name)

In [ ]:
scoring_job = list(workspace_ml_client.jobs.list(parent_job_name=job.name))[0]

workspace_ml_client.jobs.download(
    name=scoring_job.name,
    download_path=os.path.join(dataset_parent_dir, "csv-output"),
    output_name="score",
)

predictions_file = os.path.join(
    dataset_parent_dir, "csv-output", "named-outputs", "score", "predictions.csv"
)

# Load the batch predictions file with no headers into a dataframe and set your column names
score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=["row_number_per_file", "text_features", "file_name"],
)
score_df.head()

### 5.3 Test the endpoint - Using a folder of CSV files with base64 images and text samples

Invoke the batch endpoint with the input parameter pointing to the folder of CSV files containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

Note: If job failed with Out of Memory Error then please try splitting your input into smaller csv files or decreasing `mini_batch_size` for the deployment.

In [ ]:
job = None
input = Input(path=image_text_csv_folder_path, type=AssetTypes.URI_FOLDER)
num_retries = 3
for i in range(num_retries):
    try:
        job = workspace_ml_client.batch_endpoints.invoke(
            endpoint_name=endpoint.name, input=input
        )
        break
    except Exception as e:
        if i == num_retries - 1:
            raise e
        else:
            print("Endpoint invocation failed. Retrying after 5 seconds...")
            time.sleep(5)
if job is not None:
    workspace_ml_client.jobs.stream(job.name)

In [ ]:
scoring_job = list(workspace_ml_client.jobs.list(parent_job_name=job.name))[0]

workspace_ml_client.jobs.download(
    name=scoring_job.name,
    download_path=os.path.join(dataset_parent_dir, "csv-output"),
    output_name="score",
)

predictions_file = os.path.join(
    dataset_parent_dir, "csv-output", "named-outputs", "score", "predictions.csv"
)

# Load the batch predictions file with no headers into a dataframe and set your column names
score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=["row_number_per_file", "image_features", "text_features", "file_name"],
)
score_df.head()

### 6. Clean up resources - delete the endpoint
Batch endpoints use compute resources only when jobs are submitted. You can keep the batch endpoint for your reference without worrying about compute bills, or choose to delete the endpoint. If you created your compute cluster to have zero minimum instances and scale down soon after being idle, you won't be charged for an unused compute.

In [ ]:
workspace_ml_client.batch_endpoints.begin_delete(name=endpoint_name).result()